# Usando nuevo dataset

# Importando dataset

In [24]:
import jsonlines
import pandas as pd

with jsonlines.open('Dataset/SMHD/SMHD_eating_train.jsonl') as reader:
    count=0
    df = pd.DataFrame()
    for obj in reader:
        count=count+1
        lines = reader.read()
        df_inter=pd.json_normalize(lines, record_path=['posts'], meta=['id', 'label'])
        df=pd.concat([df,df_inter], axis=0)

print(count)
print(len(df['id'].unique()))
df

104
104


,body,created_utc,text,selftext,title,id,label
0,Added :),1407627843,Added :),NaN,NaN,2003202,1
1,Added :),1407627874,Added :),NaN,NaN,2003202,1
2,I lost 80lbs after moving out of my parents ho...,1407628386,I lost 80lbs after moving out of my parents ho...,NaN,NaN,2003202,1
3,I tried adding you again. Let me know if it wo...,1407631337,I tried adding you again. Let me know if it wo...,NaN,NaN,2003202,1
4,Added!,1407631412,Added!,NaN,NaN,2003202,1
...,...,...,...,...,...,...,...
95,NaN,1489089339,I'M FREEEEEEEEEEE,I'M FREEEEEEEEEEE,[ToK/EE] Just submitted my TOK essay.,8613896,0
96,NaN,1492797209,,,My first Calvin and Hobbes tattoo! What do you...,8613896,0
97,good luck with history,1494354170,good luck with history,NaN,NaN,8613896,0
98,All I've learned from this is that Comey's not...,1494981221,All I've learned from this is that Comey's not...,NaN,NaN,8613896,0


### Borramos columnas innecesarias

In [25]:
df.drop(['selftext', 'body', 'created_utc', 'title'], axis=1, inplace=True)
df

,text,id,label
0,Added :),2003202,1
1,Added :),2003202,1
2,I lost 80lbs after moving out of my parents ho...,2003202,1
3,I tried adding you again. Let me know if it wo...,2003202,1
4,Added!,2003202,1
...,...,...,...
95,I'M FREEEEEEEEEEE,8613896,0
96,,8613896,0
97,good luck with history,8613896,0
98,All I've learned from this is that Comey's not...,8613896,0


# Hacer join de los text de los mismos ids

# Tokenizado y eliminacion de stopwords

In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def preprocess(words, type='doc'):
    tokens = nltk.word_tokenize(words)
    porter = nltk.PorterStemmer()
    lemmas = [porter.stem(t) for t in tokens]
    stoplist = stopwords.words('english')
    lemmas_clean = [w for w in tokens if w not in stoplist]
    punctuation = set(string.punctuation)
    words = [w for w in lemmas_clean if  w not in punctuation]
    return words

df['text'] = df.apply(lambda row: preprocess(row['text']), axis=1)

In [27]:
df

,text,id,label
0,[Added],2003202,1
1,[Added],2003202,1
2,"[I, lost, 80lbs, moving, parents, house, mostl...",2003202,1
3,"[I, tried, adding, Let, know, worked]",2003202,1
4,[Added],2003202,1
...,...,...,...
95,"[I, 'M, FREEEEEEEEEEE]",8613896,0
96,[],8613896,0
97,"[good, luck, history]",8613896,0
98,"[All, I, 've, learned, Comey, 's, fucked]",8613896,0


# Modelizacion

In [28]:
display(df.shape)

(24317, 3)

In [39]:
import sklearn
from sklearn.model_selection import train_test_split

x = df['text']
y = df['label']

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')

x = vectorizer.fit_transform(x.apply(lambda x: ' '.join(x)))

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
y_train=y_train.astype('int')
classifier.fit(X_train_vec, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [41]:
y_pred = classifier.predict(X_test_vec)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

ValueError: X has 16973 features, but RandomForestClassifier is expecting 26772 features as input.

In [32]:
X_train_vec.shape

(17021, 26772)

In [33]:
X_test_vec.shape

(7296, 16973)

In [37]:
df.to_csv('data.csv')